We need to understand how the JS Divergence behaves as a function of beta.  
So we will generate two groups of trials, using poisson populations with fixed rates, different for each group.


In [6]:
import numpy as np
import neuraltda.simpComp as sc
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
def generate_poisson_population(ncells, ntimes, rate):
    samps = np.random.rand(ncells, ntimes)
    binmat = np.greater(rate, samps).astype(int)
    return binmat

def generate_trials(ncells, ntimes, trials_per_group, ngroups, rates):
    trials = []
    for ind, grp in enumerate(range(ngroups)):
        for t in range(trials_per_group):
            tx = generate_poisson_population(ncells, ntimes, rates[ind])
            trials.append(tx)
    return trials

def lapspectrum(trial, dim):
    ms = sc.binarytomaxsimplex(trial, rDup=True)
    scg = sc.simplicialChainGroups(ms)
    lap = sc.sparse_laplacian(scg, dim)
    lapspectra = sc.sparse_spectrum(lap)
    return lapspectra

def get_laplacian_spectra(trials, dim):
    return Parallel(n_jobs=24)(delayed(lapspectrum)(t, dim) for t in trials)
        

In [20]:
trials = generate_trials(100, 1000, 20, 2, [0.01, 0.02])
#get_laplacian_spectra(trials, 1)

In [21]:
[np.sum(t, axis=0) for t in trials]

[array([2, 2, 1, 1, 0, 1, 2, 1, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 1,
        0, 1, 3, 2, 3, 0, 0, 2, 0, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 0, 0, 2,
        0, 0, 2, 0, 0, 0, 0, 1, 2, 1, 1, 0, 0, 3, 0, 2, 2, 0, 1, 0, 1, 2,
        1, 4, 1, 0, 3, 1, 3, 0, 0, 1, 0, 2, 2, 2, 0, 0, 1, 0, 1, 0, 3, 1,
        1, 1, 1, 1, 0, 2, 1, 1, 0, 1, 3, 1, 0, 1, 5, 0, 0, 4, 0, 1, 1, 0,
        0, 0, 1, 0, 1, 2, 1, 1, 0, 1, 2, 3, 0, 0, 1, 1, 0, 1, 1, 2, 3, 0,
        0, 2, 4, 1, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 1, 2, 0, 2, 1, 3,
        1, 0, 3, 2, 0, 0, 1, 2, 0, 1, 4, 1, 1, 5, 0, 0, 1, 1, 1, 1, 3, 1,
        0, 0, 3, 0, 0, 0, 1, 0, 0, 1, 0, 5, 2, 0, 2, 0, 0, 2, 1, 0, 2, 1,
        0, 0, 1, 4, 2, 1, 0, 1, 2, 1, 3, 0, 1, 2, 1, 2, 3, 3, 0, 1, 0, 1,
        0, 1, 2, 1, 1, 1, 4, 3, 2, 0, 0, 2, 0, 0, 1, 0, 1, 0, 1, 2, 0, 2,
        1, 2, 1, 0, 0, 1, 1, 2, 1, 3, 1, 0, 2, 0, 0, 1, 2, 2, 1, 1, 0, 1,
        1, 1, 1, 0, 4, 2, 1, 1, 1, 1, 3, 0, 2, 0, 1, 1, 1, 0, 1, 4, 0, 1,
        1, 0, 1, 4, 3, 0, 1, 1, 1, 0, 